# TWEETS ANALYSIS TO DETERMINE SENTIMENTS TOWARDS VACCINATION IN AFRICA

SOURCE: ZINDI AFRICA WEEKLY CHALLENGE

POSITION ON LEADERBOARD: 10TH OF 170

NATURAL LANGUAGE PROCESSING USING NLTK LIBRARY

In [ ]:
#import needed libraries
import numpy as np;import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import nltk
import sklearn

In [ ]:
nltk.download('popular')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Package names is already up-to-date!
[nltk_data]    | Do

True

In [ ]:
#import train and test data
dir="/content/drive/My Drive"
train_data=pd.read_csv(dir+'/Zindi_vaccination/Vaccination_train.csv',na_values='NULL')
test_data=pd.read_csv(dir+'/Zindi_vaccination/Vaccination_test.csv',na_values='NULL')
train_data[:10];id=test_data.tweet_id.values

In [ ]:
from google.colab import drive
drive.mount("/content/drive",force_remount=True)

Mounted at /content/drive


There are four columns namely tweet_id,safe_text,labels,agreement
the "safe_text" column contains tweets for analysis

In [ ]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10001 entries, 0 to 10000
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   tweet_id   10001 non-null  object 
 1   safe_text  10001 non-null  object 
 2   label      10000 non-null  float64
 3   agreement  9999 non-null   float64
dtypes: float64(2), object(2)
memory usage: 312.7+ KB


In [ ]:
train_data.label.value_counts()

 0.000000    4908
 1.000000    4053
-1.000000    1038
 0.666667       1
Name: label, dtype: int64

In [ ]:
#The Labels have a missing missing, we will be dropping that row 
# Drop the row with label 0.666667
train_data=train_data[train_data['label'].notnull()]
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 10000
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   tweet_id   10000 non-null  object 
 1   safe_text  10000 non-null  object 
 2   label      10000 non-null  float64
 3   agreement  9999 non-null   float64
dtypes: float64(2), object(2)
memory usage: 390.6+ KB


In [ ]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5177 entries, 0 to 5176
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   tweet_id   5177 non-null   object
 1   safe_text  5176 non-null   object
dtypes: object(2)
memory usage: 81.0+ KB


In [ ]:
test_data[test_data.safe_text.isnull()]

,tweet_id,safe_text
2024,Dr. JAMES SHANNON,NaN


In [ ]:
#Fill Missing Test tweet with zero
test_data.fillna('vaccinate',inplace=True)

In [ ]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5177 entries, 0 to 5176
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   tweet_id   5177 non-null   object
 1   safe_text  5177 non-null   object
dtypes: object(2)
memory usage: 81.0+ KB


In [ ]:
#Extract tweets for analysis
tweets_train=train_data.safe_text.values
tweets_test=test_data.safe_text.values
print('tweets_train:',tweets_train.shape,'\n','tweets_test:',tweets_test.shape)

tweets_train: (10000,) 
 tweets_test: (5177,)


In [ ]:
tweets_test

array(['<user> <user> ... &amp; 4 a vaccine given 2 healthy peeps, FDA think just not worth the AE risk unfortunately.',
       'Students starting school without whooping cough vaccinations <url> #scpick',
       'I\'m kinda over every ep of <user> being "ripped from the headlines." Measles? Let\'s get back to crime. #SVU',
       ...,
       '<user> issues alert regarding Measles in TX. Keep your DDx up to date, people! #Emergencymedicine',
       "I can't believe people don't vaccinate their kids! I've been vaccinated for everything and then some.",
       '"<user>  Alternatives to #Flu Vaccine <url> #natural #health" A good read with a few new tips &amp; many we #jerf folk know'],
      dtype=object)

In [ ]:
#Converting all uppercase letters to lower case letters
for x,y in enumerate(tweets_train):
    tweets_train[x]=y.lower()
for x,y in enumerate(tweets_test):
    tweets_test[x]=y.lower()
print(tweets_train[:3])
print(tweets_test[:3])

['me &amp; the big homie meanboy3000 #meanboy #mb #mbs #mmr #stegmanlife @ stegman st. <url>'
 "i'm 100% thinking of devoting my career to proving autism isn't caused by vaccines due to the idiotic posts i've seen about world autism day"
 '#whatcausesautism vaccines, do not vaccinate your child']
['<user> <user> ... &amp; 4 a vaccine given 2 healthy peeps, fda think just not worth the ae risk unfortunately.'
 'students starting school without whooping cough vaccinations <url> #scpick'
 'i\'m kinda over every ep of <user> being "ripped from the headlines." measles? let\'s get back to crime. #svu']


In [ ]:
# Tokenize the tweets using Regular expression tokenizer
train=[];test=[]
for x in tweets_train:
    train.append(nltk.regexp_tokenize(x,pattern=r"#+\w+|\w+"))
for y in tweets_test:
    test.append(nltk.regexp_tokenize(y,pattern=r"#+\w+|\w+"))
test[:3]

[['user',
  'user',
  'amp',
  '4',
  'a',
  'vaccine',
  'given',
  '2',
  'healthy',
  'peeps',
  'fda',
  'think',
  'just',
  'not',
  'worth',
  'the',
  'ae',
  'risk',
  'unfortunately'],
 ['students',
  'starting',
  'school',
  'without',
  'whooping',
  'cough',
  'vaccinations',
  'url',
  '#scpick'],
 ['i',
  'm',
  'kinda',
  'over',
  'every',
  'ep',
  'of',
  'user',
  'being',
  'ripped',
  'from',
  'the',
  'headlines',
  'measles',
  'let',
  's',
  'get',
  'back',
  'to',
  'crime',
  '#svu']]

In [ ]:
#Remove all Words with length less than or equal to 3 except 'not'
a=[];b=[];train1=[];test1=[]
for x in train:
    for y in x:
        if len(y)>3 and y!='not':
            a.append(y)
    train1.append(a);a=[]
for  x in test:
    for y in x:
        if len(y)>3 and y!='not':
            b.append(y)
    test1.append(b);b=[]
test1[:3]

[['user',
  'user',
  'vaccine',
  'given',
  'healthy',
  'peeps',
  'think',
  'just',
  'worth',
  'risk',
  'unfortunately'],
 ['students',
  'starting',
  'school',
  'without',
  'whooping',
  'cough',
  'vaccinations',
  '#scpick'],
 ['kinda',
  'over',
  'every',
  'user',
  'being',
  'ripped',
  'from',
  'headlines',
  'measles',
  'back',
  'crime',
  '#svu']]

 Tokenizer pattern was defined to allow hashtag("#") before texts to allow for originality of
tweets

In [ ]:
train,test=train1,test1
train1=[];test1=[]

In [ ]:
test[:3]

[['user',
  'user',
  'vaccine',
  'given',
  'healthy',
  'peeps',
  'think',
  'just',
  'worth',
  'risk',
  'unfortunately'],
 ['students',
  'starting',
  'school',
  'without',
  'whooping',
  'cough',
  'vaccinations',
  '#scpick'],
 ['kinda',
  'over',
  'every',
  'user',
  'being',
  'ripped',
  'from',
  'headlines',
  'measles',
  'back',
  'crime',
  '#svu']]

In [ ]:
# Removing Stopwords from the text 
#1. Stopwords from default Nltk corpus package
a=[];b=[]
sw=nltk.corpus.stopwords.words('english')
for x in train:
    for y in x:
        if y not in sw:
            a.append(y)
    train1.append(a);a=[]
for x in test:
    for y in x:
        if y not in sw:
            b.append(y)
    test1.append(b);b=[]
train,test=train1,test1;train1=[];test1=[]
test[:3]

[['user',
  'user',
  'vaccine',
  'given',
  'healthy',
  'peeps',
  'think',
  'worth',
  'risk',
  'unfortunately'],
 ['students',
  'starting',
  'school',
  'without',
  'whooping',
  'cough',
  'vaccinations',
  '#scpick'],
 ['kinda',
  'every',
  'user',
  'ripped',
  'headlines',
  'measles',
  'back',
  'crime',
  '#svu']]

In [ ]:
 #2. Custom stopwords from Text source link
sw=[x.strip() for x in open(dir+'/Stop_words/stopwords.txt')]
for x in train:
    for y in x:
        if y not in sw:
            a.append(y)
    train1.append(a);a=[]
for x in test:
    for y in x:
        if y not in sw:
            b.append(y)
    test1.append(b);b=[]
train,test=train1,test1;train1=[];test1=[]
test[:5]

[['user',
  'user',
  'vaccine',
  'healthy',
  'peeps',
  'worth',
  'risk',
  'unfortunately'],
 ['students',
  'starting',
  'school',
  'whooping',
  'cough',
  'vaccinations',
  '#scpick'],
 ['kinda', 'user', 'ripped', 'headlines', 'measles', 'crime', '#svu'],
 ['innocent',
  'children',
  'lack',
  'vaccination',
  'million',
  'parents',
  '#sb277'],
 ['eyeing',
  'bird',
  'vaccine',
  'humans',
  'risk',
  'federal',
  'officials',
  'wednesday',
  'taking',
  'steps']]

In [ ]:
# 3. stopwords from source link
sw=[x.strip() for x in open(dir+'/Stop_words/srop_words_english.txt')]
for x in train:
    for y in x:
        if y not in sw:
            a.append(y)
    train1.append(a);a=[]
for x in test:
    for y in x:
        if y not in sw:
            b.append(y)
    test1.append(b);b=[]
train,test=train1,test1;train1=[];test1=[]
test[:5]

[['user', 'user', 'vaccine', 'healthy', 'peeps', 'worth', 'risk'],
 ['students',
  'starting',
  'school',
  'whooping',
  'cough',
  'vaccinations',
  '#scpick'],
 ['kinda', 'user', 'ripped', 'headlines', 'measles', 'crime', '#svu'],
 ['innocent', 'children', 'lack', 'vaccination', 'parents', '#sb277'],
 ['eyeing',
  'bird',
  'vaccine',
  'humans',
  'risk',
  'federal',
  'officials',
  'wednesday',
  'steps']]

In [ ]:
# Using Porter Stemmer from NLTK module
stem=nltk.PorterStemmer()
for x in train:
    for y in x:
        a.append(stem.stem(y))
    train1.append(a);a=[]
for x in test:
    for y in x:
        b.append(stem.stem(y))
    test1.append(b);b=[]
train,test=train1,test1

In [ ]:
test[:5]

[['user', 'user', 'vaccin', 'healthi', 'peep', 'worth', 'risk'],
 ['student', 'start', 'school', 'whoop', 'cough', 'vaccin', '#scpick'],
 ['kinda', 'user', 'rip', 'headlin', 'measl', 'crime', '#svu'],
 ['innoc', 'children', 'lack', 'vaccin', 'parent', '#sb277'],
 ['eye',
  'bird',
  'vaccin',
  'human',
  'risk',
  'feder',
  'offici',
  'wednesday',
  'step']]

In [ ]:
#Transform Train and Test data back into strings
train1=[];test1=[]
for x in train:
    train1.append(' '.join(x))
for y in test:
    test1.append(' '.join(y))
train1[:3]

['homi meanboy3000 #meanboy #mb #mmr #stegmanlif stegman',
 'think devot career prove autism caus vaccin idiot post autism',
 '#whatcausesaut vaccin vaccin child']

MODEL BUILDING

In [ ]:
#Combine Train and Test data To fit on CountVectorizer
T=train1.copy()
for x in test1:
    T.append(x)
print(T[-3:])

['user issu alert measl peopl #emergencymedicin', 'peopl vaccin kid vaccin', 'user altern #flu vaccin #natur #health read tip #jerf folk']


In [ ]:
#Using CountVectorizer from sklearn
cv=sklearn.feature_extraction.text.CountVectorizer(token_pattern=r"#+\w+|\w+")
cv.fit(T)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='#+\\w+|\\w+', tokenizer=None,
                vocabulary=None)

In [ ]:
train=cv.transform(train1)
test=cv.transform(test1)
train.shape

(10000, 12455)

In [ ]:
print(train.toarray())

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [ ]:
train

<10000x12455 sparse matrix of type '<class 'numpy.int64'>'
	with 69259 stored elements in Compressed Sparse Row format>

In [ ]:
y_train=train_data.label.values
y_train.shape

(10000,)

RANDOM FOREST REGRESSOR

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [44]:
from sklearn.model_selection import cross_val_predict

In [50]:
from sklearn.metrics import mean_squared_error

In [45]:
 
forest_reg=RandomForestRegressor(n_estimators=70,max_leaf_nodes=50,min_samples_split=10,min_samples_leaf=5)
forest_reg.fit(train,y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=50,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=5,
                      min_samples_split=10, min_weight_fraction_leaf=0.0,
                      n_estimators=70, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [60]:
forest_pre=cross_val_predict(forest_reg,train,y_train,cv=5)
np.sqrt(mean_squared_error(y_train,forest_pre))

0.5863310181421905

LINEAR SUPPORT VECTOR REGRESSOR

In [51]:
from sklearn.svm import LinearSVR

In [57]:
#Trying different Paramaters for C and loss
n=[0.001,0.01,0.025,0.1,1,5]
L=['epsilon_insensitive','squared_epsilon_insensitive']
for x in n:
    for y in L:
        svm_reg=LinearSVR(random_state=42,max_iter=10000,C=x,loss=y)
        svm_pre=cross_val_predict(svm_reg,train,y_train,cv=5)
        print(x,y,round(np.sqrt(mean_squared_error(y_train,svm_pre)),3))

0.001 epsilon_insensitive 0.631
0.001 squared_epsilon_insensitive 0.607
0.01 epsilon_insensitive 0.607
0.01 squared_epsilon_insensitive 0.581
0.025 epsilon_insensitive 0.599
0.025 squared_epsilon_insensitive 0.577
0.1 epsilon_insensitive 0.605
0.1 squared_epsilon_insensitive 0.587
1 epsilon_insensitive 0.693
1 squared_epsilon_insensitive 0.664


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


5 epsilon_insensitive 0.818
5 squared_epsilon_insensitive 0.773


In [58]:
svm_reg=LinearSVR(random_state=42,max_iter=10000,C=0.025,loss='squared_epsilon_insensitive')
svm_reg.fit(train,y_train)

LinearSVR(C=0.025, dual=True, epsilon=0.0, fit_intercept=True,
          intercept_scaling=1.0, loss='squared_epsilon_insensitive',
          max_iter=10000, random_state=42, tol=0.0001, verbose=0)

In [59]:
sub_file=pd.DataFrame()
sub_file['tweet_id']=id
sub_file['label']=svm_reg.predict(test)
sub_file

,tweet_id,label
0,00BHHHP1,0.181554
1,00UNMD0E,0.585760
2,01AXPTJF,0.077253
3,01HOEQJW,0.527812
4,01JUKMAO,0.286036
...,...,...
5172,ZXVVNC5O,0.943001
5173,ZYIANVI8,0.148038
5174,ZYITEHAH,0.217119
5175,ZZ3BMBTG,0.789997
